In [ ]:
!pip install huggingface_hub

In [2]:
from huggingface_hub import snapshot_download

In [ ]:
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
snapshot_download(repo_id=model_id, local_dir="Tinyllama-hf",
                  local_dir_use_symlinks=False, revision="main")

In [4]:
!ls -lash Tinyllama-hf

total 2.1G
4.0K drwxr-xr-x 2 root root 4.0K Jan 14 16:25 .
4.0K drwxr-xr-x 1 root root 4.0K Jan 14 16:25 ..
4.0K -rw-r--r-- 1 root root  608 Jan 14 16:25 config.json
4.0K -rw-r--r-- 1 root root  566 Jan 14 16:25 eval_results.json
4.0K -rw-r--r-- 1 root root  124 Jan 14 16:25 generation_config.json
4.0K -rw-r--r-- 1 root root 1.5K Jan 14 16:25 .gitattributes
2.1G -rw-r--r-- 1 root root 2.1G Jan 14 16:25 model.safetensors
4.0K -rw-r--r-- 1 root root 3.1K Jan 14 16:25 README.md
4.0K -rw-r--r-- 1 root root  551 Jan 14 16:25 special_tokens_map.json
4.0K -rw-r--r-- 1 root root 1.3K Jan 14 16:25 tokenizer_config.json
1.8M -rw-r--r-- 1 root root 1.8M Jan 14 16:25 tokenizer.json
492K -rw-r--r-- 1 root root 489K Jan 14 16:25 tokenizer.model


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git

In [ ]:
###Need to restart the system after this
!pip install -r llama.cpp/requirements.txt


In [1]:
!python llama.cpp/convert.py -h

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/content/llama.cpp/gguf-py
usage: convert.py [-h] [--awq-path AWQ_PATH] [--dump] [--dump-single] [--vocab-only]
                  [--outtype {f32,f16,q8_0}] [--vocab-dir VOCAB_DIR] [--vocab-type {spm,bpe,hfft}]
                  [--pad-vocab] [--outfile OUTFILE] [--ctx CTX] [--concurrency CONCURRENCY]
                  [--big-endian]
                  model

Convert a LLaMa model to a GGML compatible file

positional arguments:
  model                 Directory containing the model file or the model file itself (*.pth, *.pt,
                        *.bin)

options:
  -h, --help            show this help message and exit
  --awq-path AWQ_PATH   Path to the Activation-aware Weight Quantization cache file
  --dump                Display the m

In [2]:
!python llama.cpp/convert.py Tinyllama-hf \
  --outfile Tinyllama-1b-v1.0.gguf \
  --outtype q8_0

/content/llama.cpp/gguf-py
Loading model file Tinyllama-hf/model.safetensors
params = Params(n_vocab=32000, n_embd=2048, n_layer=22, n_ctx=2048, n_ff=5632, n_head=32, n_head_kv=4, f_norm_eps=1e-05, n_experts=None, n_experts_used=None, rope_scaling_type=None, f_rope_freq_base=10000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyQ8_0: 7>, path_model=PosixPath('Tinyllama-hf'))
Loading vocab file 'Tinyllama-hf/tokenizer.model', type 'spm'
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting layer 12
Permuting layer 13
Permuting layer 14
Permuting layer 15
Permuting layer 16
Permuting layer 17
Permuting layer 18
Permuting layer 19
Permuting layer 20
Permuting layer 21
lm_head.weight                                   -> output.weight                            | BF16   | [32000, 20

In [3]:
!ls -lash Tinyllama-1b-v1.0.gguf

1.1G -rw-r--r-- 1 root root 1.1G Jan 14 16:32 Tinyllama-1b-v1.0.gguf


In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "epsil/Tinyllama-1b-v1.0-gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj="Tinyllama-1b-v1.0.gguf",
    path_in_repo="Tinyllama-1b-v1.0.gguf",
    repo_id=model_id,
)

### Evaluation of Model

In [ ]:
!pip install ctransformers[cuda]

In [14]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("epsil/Tinyllama-1b-v1.0-gguf", model_file="Tinyllama-1b-v1.0.gguf")

print(llm("AI is going to"))


Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Tinyllama-1b-v1.0.gguf:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

 make our lives better? I’m excited about the possibilities. Can you help me understand more about how it works and what specific industries it could be used in? I’m a marketer, so I want to know how this technology can benefit my business.
